<a href="https://colab.research.google.com/github/Username1234jj/AI-Research-Assistant/blob/main/AI_Research_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Reasearch Assistant - **Setup**

In [ ]:
# 🧠 AI Research Assistant - Setup

!pip install PyMuPDF transformers gradio torch torchvision torchaudio --quiet

import fitz  # PyMuPDF
from transformers import pipeline
import gradio as gr

print("✅ All libraries installed and imported successfully!")


# Upload and Extract Text from **PDF**

In [ ]:
# 🧾 Step 2 - Upload and Extract Text from PDF
from google.colab import files
import fitz  # PyMuPDF

print("📤 Please upload your research paper (PDF)...")
uploaded = files.upload()

pdf_text = ""
for filename in uploaded.keys():
    with fitz.open(filename) as pdf:
        for page in pdf:
            pdf_text += page.get_text()

print(f"✅ Extracted {len(pdf_text)} characters from {list(uploaded.keys())[0]}")


📤 Please upload your research paper (PDF)...


Saving research paper.pdf to research paper.pdf
✅ Extracted 68589 characters from research paper.pdf


# Summarize the Research Paper using AI

In [ ]:
# 🤖 Step 3 - Summarize the Research Paper Text
!pip install transformers --quiet

from transformers import pipeline

# Load a summarization model (free + works on Colab CPU)
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Split text into manageable chunks
max_chunk = 1000
chunks = [pdf_text[i:i+max_chunk] for i in range(0, len(pdf_text), max_chunk)]

summary = ""
for i, chunk in enumerate(chunks):
    print(f"🧩 Summarizing chunk {i+1}/{len(chunks)} ...")
    out = summarizer(chunk, max_length=200, min_length=50, do_sample=False)
    summary += out[0]['summary_text'] + " "

print("\n✅ Summary Generated Successfully!\n")
print(summary[:3000])  # Print first part of summary


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

🧩 Summarizing chunk 15/69 ...
🧩 Summarizing chunk 16/69 ...
🧩 Summarizing chunk 17/69 ...
🧩 Summarizing chunk 18/69 ...
🧩 Summarizing chunk 19/69 ...


Your max_length is set to 200, but your input_length is only 193. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=96)


🧩 Summarizing chunk 20/69 ...
🧩 Summarizing chunk 21/69 ...
🧩 Summarizing chunk 22/69 ...
🧩 Summarizing chunk 23/69 ...
🧩 Summarizing chunk 24/69 ...
🧩 Summarizing chunk 25/69 ...
🧩 Summarizing chunk 26/69 ...
🧩 Summarizing chunk 27/69 ...
🧩 Summarizing chunk 28/69 ...
🧩 Summarizing chunk 29/69 ...
🧩 Summarizing chunk 30/69 ...
🧩 Summarizing chunk 31/69 ...
🧩 Summarizing chunk 32/69 ...
🧩 Summarizing chunk 33/69 ...
🧩 Summarizing chunk 34/69 ...
🧩 Summarizing chunk 35/69 ...
🧩 Summarizing chunk 36/69 ...
🧩 Summarizing chunk 37/69 ...
🧩 Summarizing chunk 38/69 ...
🧩 Summarizing chunk 39/69 ...
🧩 Summarizing chunk 40/69 ...
🧩 Summarizing chunk 41/69 ...
🧩 Summarizing chunk 42/69 ...
🧩 Summarizing chunk 43/69 ...
🧩 Summarizing chunk 44/69 ...
🧩 Summarizing chunk 45/69 ...
🧩 Summarizing chunk 46/69 ...
🧩 Summarizing chunk 47/69 ...
🧩 Summarizing chunk 48/69 ...
🧩 Summarizing chunk 49/69 ...
🧩 Summarizing chunk 50/69 ...
🧩 Summarizing chunk 51/69 ...
🧩 Summarizing chunk 52/69 ...
🧩 Summariz

Your max_length is set to 200, but your input_length is only 156. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=78)


🧩 Summarizing chunk 69/69 ...

✅ Summary Generated Successfully!

AsyncThink is a new paradigm of reasoning with large language models. It organizes the internalthinking process into concurrently executable structures. AsyncThink achieves 28% lower inference latency compared to parallelthinking. More importantly, the thinking structure can be further optimized through reinforcement learning. AsyncThink learns to form an agentic organization to think concurrently and collaboratively. Sequential thinking employs a purely sequential decoding trajectory; parallel thinkingexecutes multiple independent traces with an outcome aggregation. AsyncThink generalizes its learned asynchronous thinking capabilities, effectively tackling unseen tasks without additional training. Large language models have unlocked remarkable reasoning capabilities as individual agents. realizing this vision requires reasoning models that can not only think independently but also think collaboratively as an organized s

# Ask Questions About the Paper (Q&A System)

In [7]:
# 💬 Step 4 - Ask Questions About the Paper
!pip install transformers --quiet

from transformers import pipeline

# Load a question-answering model
qa_model = pipeline("question-answering", model="deepset/roberta-base-squad2")

print("🤖 Your AI Research Assistant is ready!")
print("You can now ask questions about the paper.\n")

while True:
    question = input("❓ Ask a question (or type 'exit' to stop): ")
    if question.lower() == "exit":
        print("👋 Session ended. Great job, researcher!")
        break
    result = qa_model(question=question, context=pdf_text)
    print(f"💡 Answer: {result['answer']}\n")


Device set to use cpu


🤖 Your AI Research Assistant is ready!
You can now ask questions about the paper.

❓ Ask a question (or type 'exit' to stop): What is the main idea of this paper?
💡 Answer: scaling the quantity of workers

❓ Ask a question (or type 'exit' to stop): exit
👋 Session ended. Great job, researcher!


# **Create a Web App Interface with Gradio**

In [8]:
# 🌐 Step 5 - Web App for the AI Research Assistant
!pip install gradio --quiet
import gradio as gr
from transformers import pipeline

# Load models
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
qa_model = pipeline("question-answering", model="deepset/roberta-base-squad2")

def summarize_paper(pdf_text):
    max_chunk = 1000
    chunks = [pdf_text[i:i+max_chunk] for i in range(0, len(pdf_text), max_chunk)]
    summary = ""
    for chunk in chunks:
        out = summarizer(chunk, max_length=200, min_length=50, do_sample=False)
        summary += out[0]['summary_text'] + " "
    return summary

def answer_question(question, pdf_text):
    if not question.strip():
        return "Please type a question."
    result = qa_model(question=question, context=pdf_text)
    return result['answer']

# Define Gradio interface
with gr.Blocks() as ai_research_assistant:
    gr.Markdown("# 🤖 AI Research Assistant")
    gr.Markdown("Upload a research paper, summarize it, and ask questions!")

    pdf_input = gr.File(label="Upload your research paper (PDF)")
    summary_output = gr.Textbox(label="Summary", lines=10)
    question_input = gr.Textbox(label="Ask a question about the paper")
    answer_output = gr.Textbox(label="Answer")

    summarize_btn = gr.Button("Summarize Paper")
    ask_btn = gr.Button("Ask Question")

    pdf_state = gr.State("")

    def extract_text_from_pdf(pdf_file):
        import fitz
        text = ""
        with fitz.open(pdf_file.name) as pdf:
            for page in pdf:
                text += page.get_text()
        return text

    summarize_btn.click(fn=lambda f: summarize_paper(extract_text_from_pdf(f)), inputs=pdf_input, outputs=summary_output)
    ask_btn.click(fn=lambda q, f: answer_question(q, extract_text_from_pdf(f)), inputs=[question_input, pdf_input], outputs=answer_output)

ai_research_assistant.launch()


Device set to use cpu
Device set to use cpu


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8b5ed62c848a117ba4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
